In [35]:
import pandas as pd
import re
import underthesea
from string import punctuation
import warnings
import numpy as np
punctuation = punctuation + '‘’“”'
vietnamese_words = 'aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ'
warnings.filterwarnings('ignore')

In [2]:
def read_data(path):
    doc = open(path, encoding='utf-8').read().split('\n')
    text = []
    label = []
    i = 1
    while i < len(doc):
        text.append(doc[i])
        label.append(doc[i + 1])
        i += 4

    df = pd.DataFrame({'review': text, 'sentiment': label})
    return df


In [3]:
def adding_label(data):
    label = data["sentiment"].values

    clean_label = []

    for i in range(len(label)):
        sample = label[i].split(",")
        for j in range(len(sample)):
            sample[j] = re.sub("[{}]", "", sample[j].strip())
        clean_label.append(sample)

    y = []
    for i in range(len(clean_label)):
        row = clean_label[i]

        j = 0
        while j < len(row):
            if(row[j+1] == 'positive'):
                y.append([i + 1 , row[j], 1])
            elif(row[j+1] == 'negative'):
                y.append([i + 1 , row[j], -1])
            else:
                y.append([i + 1 , row[j], 0])
            j += 2
    return y

In [4]:
def data_with_sparse_matrix(df):
    all_classes = ["Text"]
    all_classes += list(set([i[1] for i in adding_label(df)]))
    y = adding_label(df)
    new_dat = pd.DataFrame(columns=all_classes)
    all_classes.remove("Text")
    new_dat.Text = df.review
    new_dat[all_classes] = 0
    sparse_matrix = np.zeros((len(df), len(all_classes)))
    dict_class = {}
    index = 0
    for i in all_classes:
        dict_class[i] = index
        index += 1
    sparse_matrix = list(sparse_matrix.astype(int))
    for i in y:
        sparse_matrix[i[0] - 1][dict_class[i[1]]] = i[2]
    for i in y:
        new_dat[i[1]][i[0] - 1] = i[2]
    final_dat = pd.DataFrame({"Text": new_dat.Text, "Label": sparse_matrix})

    data_with_all_classes = new_dat
    data_with_sparse = final_dat
    
    return data_with_sparse, data_with_all_classes

In [5]:
df = read_data('Dataset_For_Work\\1-VLSP2018-SA-Hotel-train (7-3-2018).txt')

In [6]:
df.head(10)

,review,sentiment
0,Rộng rãi KS mới nhưng rất vắng. Các dịch vụ ch...,"{HOTEL#DESIGN&FEATURES, positive}, {HOTEL#GENE..."
1,"Địa điểm thuận tiện, trong vòng bán kính 1,5km...","{LOCATION#GENERAL, positive}"
2,"Phục vụ, view đẹp, vị trí","{SERVICE#GENERAL, positive}, {HOTEL#GENERAL, p..."
3,"thuận tiện , sạch sẽ , vui vẻ hài lòng","{HOTEL#COMFORT, positive}, {HOTEL#CLEANLINESS,..."
4,Vị trí đẹp; Có quán bar view đẹp; Nhân viên th...,"{LOCATION#GENERAL, positive}, {FACILITIES#GENE..."
5,- Co view huong Ho tay- sach se-nhan vien tan ...,"{HOTEL#GENERAL, positive}, {HOTEL#CLEANLINESS,..."
6,"Phòng ốc sạch, giường thoải mái, nhân viên thâ...","{ROOMS#CLEANLINESS, positive}, {ROOM_AMENITIES..."
7,"gần Hồ Tây, view nhìn ra hồ lãng mạn, đi bộ dọ...","{LOCATION#GENERAL, positive}, {HOTEL#GENERAL, ..."
8,Hình thức không thể quyết định nội dung.H&atil...,"{HOTEL#COMFORT, positive}"
9,Tôi ở đây lần này là lần thứ 4. Khi nhận phòng...,"{ROOMS#CLEANLINESS, negative}, {ROOM_AMENITIES..."


In [9]:
all_classes = list(set([i[1] for i in adding_label(df)]))
y = adding_label(df)

In [10]:
all_classes[:10]

['ROOM_AMENITIES#MISCELLANEOUS',
 'ROOMS#DESIGN&FEATURES',
 'SERVICE#GENERAL',
 'FOOD&DRINKS#PRICES',
 'ROOM_AMENITIES#COMFORT',
 'ROOMS#COMFORT',
 'HOTEL#PRICES',
 'HOTEL#COMFORT',
 'FACILITIES#PRICES',
 'FACILITIES#QUALITY']

In [11]:
y[:10]

[[1, 'HOTEL#DESIGN&FEATURES', 1],
 [1, 'HOTEL#GENERAL', -1],
 [2, 'LOCATION#GENERAL', 1],
 [3, 'SERVICE#GENERAL', 1],
 [3, 'HOTEL#GENERAL', 1],
 [3, 'LOCATION#GENERAL', 1],
 [4, 'HOTEL#COMFORT', 1],
 [4, 'HOTEL#CLEANLINESS', 1],
 [4, 'SERVICE#GENERAL', 1],
 [5, 'LOCATION#GENERAL', 1]]

In [12]:
final_dat,new_dat = data_with_sparse_matrix(df)

In [ ]:
def clean_text(df,option=0):

    stop_words = open('Dataset_For_Work\\vietnamese-stopwords.txt','r',encoding='utf-8').read().split('\n')
    for i in range(len(df.Text)):

        if(option==1):
            df.Text[i] = re.sub(f'[^{vietnamese_words}{punctuation}]',' ',df.Text[i]) #Việc này để loại bỏ các ký tự biểu tượng cảm xúc trong review Restaurant

        df.Text[i] = underthesea.sent_tokenize(df.Text[i])

    for i in range(len(df.Text)):
        for j in range(len(df.Text[i])):

            df.Text[i][j] = re.sub(f'[{punctuation}]',' ',df.Text[i][j])

            for s_w in stop_words:
                df.Text[i][j] = re.sub(f' {s_w} ',' ',df.Text[i][j])

    return df

In [14]:
test = clean_text(final_dat)

In [ ]:
final_dat.head() #Đây là cái để đem đi train

,Text,Label
0,Rộng rãi KS mới nhưng rất vắng. Các dịch vụ ch...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1,..."
1,"Địa điểm thuận tiện, trong vòng bán kính 1,5km...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"Phục vụ, view đẹp, vị trí","[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,"thuận tiện , sạch sẽ , vui vẻ hài lòng","[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Vị trí đẹp; Có quán bar view đẹp; Nhân viên th...,"[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [53]:
final_dat = test
final_dat.head()

,Text,Label
0,"[Rộng rãi KS vắng , Các dịch vụ chất ]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0,..."
1,[Địa thuận tiện vòng kính 1 5km quán ngon],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,[Phục vụ view đẹp vị trí],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
3,[thuận tiện sạch vui vẻ hài lòng],"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,[Vị trí đẹp Có quán bar view đẹp Nhân viên t...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
new_dat.head() #Cái này làm thêm để nhìn cho đẹp

,Text,HOTEL#CLEANLINESS,LOCATION#GENERAL,FOOD&DRINKS#STYLE&OPTIONS,ROOM_AMENITIES#DESIGN&FEATURES,HOTEL#MISCELLANEOUS,FACILITIES#GENERAL,SERVICE#GENERAL,ROOM_AMENITIES#COMFORT,FACILITIES#COMFORT,...,ROOMS#CLEANLINESS,HOTEL#DESIGN&FEATURES,ROOMS#GENERAL,FACILITIES#DESIGN&FEATURES,ROOMS#DESIGN&FEATURES,FOOD&DRINKS#MISCELLANEOUS,FACILITIES#CLEANLINESS,ROOM_AMENITIES#CLEANLINESS,FACILITIES#PRICES,ROOMS#MISCELLANEOUS
0,Rộng rãi KS mới nhưng rất vắng. Các dịch vụ ch...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,"Địa điểm thuận tiện, trong vòng bán kính 1,5km...",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Phục vụ, view đẹp, vị trí",0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"thuận tiện , sạch sẽ , vui vẻ hài lòng",1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Vị trí đẹp; Có quán bar view đẹp; Nhân viên th...,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
